### gold.report_products View

This Spark SQL script creates a view named `gold.report_products` in the `gold` schema. The view aggregates product data from the `gold.fact_sales` and `gold.dim_products` tables to provide a detailed report on product performance, including sales metrics, customer reach, and segmentation.

#### Purpose
The `gold.report_products` view is designed to:
- Summarize product-level metrics such as total orders, total sales, and customer counts.
- Segment products based on total sales (High-Performer, Mid-Range, Low-Performer).
- Calculate derived metrics like average selling price, average order revenue (AOR), and average monthly revenue.
- Provide insights into product recency and lifespan for inventory and sales analysis.

#### Schema
The view is created in the `gold` schema and relies on the following tables:
- `gold.fact_sales`: Contains sales transaction data (`order_number`, `order_date`, `customer_key`, `sales_amount`, `quantity`, `product_key`).
- `gold.dim_products`: Contains product details (`product_key`, `product_name`, `category`, `subcategory`, `cost`).

#### Structure
The view is built using two Common Table Expressions (CTEs) and a final SELECT statement:

1. **base_query CTE**
   - **Purpose**: Retrieves core columns from `gold.fact_sales` and `gold.dim_products`.
   - **Joins**: Left joins `fact_sales` with `dim_products` on `product_key`.
   - **Filters**: Excludes rows where `order_date` is NULL.
   - **Columns**:
     - `order_number`: Unique identifier for each order.
     - `order_date`: Date of the order.
     - `customer_key`: Unique identifier for the customer.
     - `sales_amount`: Total sales amount for the order.
     - `quantity`: Number of items in the order.
     - `product_key`: Unique identifier for the product.
     - `product_name`: Name of the product.
     - `category`: Product category.
     - `subcategory`: Product subcategory.
     - `cost`: Cost of the product.

2. **product_aggregations CTE**
   - **Purpose**: Aggregates data at the product level.
   - **Grouping**: Groups by `product_key`, `product_name`, `category`, `subcategory`, and `cost`.
   - **Metrics**:
     - `lifespan`: Months between first and last sale, calculated using `FLOOR(MONTHS_BETWEEN(MAX(order_date), MIN(order_date)))`.
     - `last_sale_date`: Most recent sale date.
     - `total_orders`: Count of distinct orders.
     - `total_customers`: Count of distinct customers.
     - `total_sales`: Sum of sales amounts.
     - `total_quantity`: Sum of quantities sold.
     - `avg_selling_price`: Average sales amount per unit, rounded to 1 decimal place, using `ROUND(AVG(sales_amount / NULLIF(quantity, 0)), 1)`.

3. **Final SELECT**
   - **Purpose**: Builds the final report with additional derived fields and product segmentation.
   - **Columns**:
     - `product_key`, `product_name`, `category`, `subcategory`, `cost`: Direct from `product_aggregations`.
     - `last_sale_date`: Most recent sale date.
     - `recency_in_months`: Months since the last sale, calculated using `FLOOR(MONTHS_BETWEEN(CURRENT_DATE(), last_sale_date))`.
     - `product_segment`: Segments products based on total sales:
       - `High-Performer`: Total sales > 50,000.
       - `Mid-Range`: Total sales ≥ 10,000 and ≤ 50,000.
       - `Low-Performer`: Total sales < 10,000.
     - `lifespan`, `total_orders`, `total_sales`, `total_quantity`, `total_customers`, `avg_selling_price`: Direct from `product_aggregations`.
     - `avg_order_revenue`: Total sales divided by total orders (returns 0 if `total_orders` is 0).
     - `avg_monthly_revenue`: Total sales divided by lifespan (returns `total_sales` if `lifespan` is 0).

In [0]:
# Drop the existing view if it exists
spark.sql("DROP VIEW IF EXISTS dwh_project.gold.report_products")

# Create the new view
spark.sql("""
CREATE VIEW dwh_project.gold.report_products AS
WITH base_query AS (
    SELECT
        f.order_number,
        f.order_date,
        f.customer_key,
        f.sales_amount,
        f.quantity,
        p.product_key,
        p.product_name,
        p.category,
        p.subcategory,
        p.cost
    FROM dwh_project.gold.fact_sales f
    LEFT JOIN dwh_project.gold.dim_products p
        ON f.product_key = p.product_key
    WHERE order_date IS NOT NULL
),

product_aggregations AS (
    SELECT
        product_key,
        product_name,
        category,
        subcategory,
        cost,
        CAST(
            MONTHS_BETWEEN(MAX(order_date), MIN(order_date))
            AS INT
        ) AS lifespan,
        MAX(order_date) AS last_sale_date,
        COUNT(DISTINCT order_number) AS total_orders,
        COUNT(DISTINCT customer_key) AS total_customers,
        SUM(sales_amount) AS total_sales,
        SUM(quantity) AS total_quantity,
        ROUND(AVG(CAST(sales_amount AS DOUBLE) / NULLIF(quantity, 0)), 1) AS avg_selling_price
    FROM base_query
    GROUP BY
        product_key,
        product_name,
        category,
        subcategory,
        cost
)

SELECT 
    product_key,
    product_name,
    category,
    subcategory,
    cost,
    last_sale_date,
    CAST(
        MONTHS_BETWEEN(CURRENT_DATE, last_sale_date)
        AS INT
    ) AS recency_in_months,
    CASE
        WHEN total_sales > 50000 THEN 'High-Performer'
        WHEN total_sales >= 10000 THEN 'Mid-Range'
        ELSE 'Low-Performer'
    END AS product_segment,
    lifespan,
    total_orders,
    total_sales,
    total_quantity,
    total_customers,
    avg_selling_price,
    CASE 
        WHEN total_orders = 0 THEN 0
        ELSE total_sales / total_orders
    END AS avg_order_revenue,
    CASE
        WHEN lifespan = 0 THEN total_sales
        ELSE total_sales / lifespan
    END AS avg_monthly_revenue
FROM product_aggregations
""")

# Display the new view
df = spark.sql("SELECT * FROM dwh_project.gold.report_products")
display(df)

product_key,product_name,category,subcategory,cost,last_sale_date,recency_in_months,product_segment,lifespan,total_orders,total_sales,total_quantity,total_customers,avg_selling_price,avg_order_revenue,avg_monthly_revenue
121,Mountain-200 Black- 42,Bikes,Mountain Bikes,1252,2013-12-28,140,High-Performer,23,614,1363128,614,604,2220.1,2220.0781758957655,59266.434782608696
128,Road-750 Black- 52,Bikes,Road Bikes,344,2013-12-28,140,High-Performer,12,386,208440,386,386,540.0,540.0,17370.0
19,Road-150 Red- 56,Bikes,Road Bikes,2171,2011-12-27,164,High-Performer,11,295,1055510,295,295,3578.0,3578.0,95955.45454545454
282,LL Mountain Tire,Accessories,Tires and Tubes,9,2014-01-28,139,Mid-Range,12,860,21550,862,828,25.0,25.058139534883722,1795.8333333333333
45,Road-650 Red- 58,Bikes,Road Bikes,487,2012-12-18,152,High-Performer,22,74,56346,74,74,761.4,761.4324324324324,2561.181818181818
141,Road-250 Black- 52,Bikes,Road Bikes,1555,2013-12-25,140,High-Performer,23,319,734425,319,312,2302.3,2302.2727272727275,31931.521739130436
109,Mountain-500 Black- 48,Bikes,Mountain Bikes,295,2013-12-26,140,Mid-Range,11,56,30240,56,56,540.0,540.0,2749.090909090909
119,Mountain-400-W Silver- 46,Bikes,Mountain Bikes,420,2013-12-24,140,High-Performer,11,138,106122,138,138,769.0,769.0,9647.454545454546
107,Mountain-500 Black- 42,Bikes,Mountain Bikes,295,2013-12-25,140,Mid-Range,11,49,26460,49,49,540.0,540.0,2405.4545454545455
163,Touring-1000 Yellow- 50,Bikes,Touring Bikes,1482,2013-12-23,140,High-Performer,11,151,359984,151,150,2384.0,2384.0,32725.81818181818
